In [1]:
# Cell 1: 导入必要的库
import random
import json
from typing import List, Dict
from sklearn.metrics import f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer

f:\anaconda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Cell 2: 模型初始化 (只需运行一次)
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype="auto",
    # device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
# Cell 3: 数据处理函数定义
def convert_to_raw_text(input_file: str, output_file: str):
    """将带BIO标注的文件转换为纯文本文件"""
    sentences = []
    current_sentence = []
    
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                char, _ = line.split('\t')
                current_sentence.append(char)
            elif current_sentence:
                sentences.append(''.join(current_sentence))
                current_sentence = []
    
    if current_sentence:
        sentences.append(''.join(current_sentence))
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')

def sample_sentences(file_path: str, n: int = 10) -> List[str]:
    """从文件中随机抽取n个句子"""
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = f.readlines()
    return random.sample(sentences, min(n, len(sentences)))

def read_true_labels(file_path: str) -> List[List[str]]:
    """从原始文件读取真实标签"""
    sentences_labels = []
    current_labels = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                _, label = line.split('\t')
                current_labels.append(label)
            elif current_labels:
                sentences_labels.append(current_labels)
                current_labels = []
    
    if current_labels:
        sentences_labels.append(current_labels)
    
    return sentences_labels

In [6]:
# Cell 4: Prompt模板定义
PROMPT_TEMPLATES = {
    'PER': """你是一个专业的古文命名实体识别专家。请仔细阅读以下古文内容，在保证理解意思的基础上，帮我识别出其中的人名。人名可能有多种形式：可能是人的名、字、号、谥号、封号等。尤其注意，人名可能以单字出现，请注意识别。

常见的名字有：至德、高宗、恭、宣帝、太宗、珪、太宗、子智、子安世。这只是其中的一些例子，可能还有更多别的名字需要你去发现，请注意识别。
需要注意，不要把地名、官职名、书名等识别为人名。

以下内容为一个输入输出示例:
输入: "至德为并州刺史"
输出:
至德

请识别下面文本中的人名:
{text}
""",
    
    'LOC': """你是一个专业的古文命名实体识别专家。请仔细阅读以下古文内容，帮我识别出其中的地名。地名包括：国名、州名、郡名、县名、山名、水名等。

常见的地名有：并州、晋州、晋阳、长安、洛阳、天台山、至真观。这只是其中的一些例子，可能还有更多别的地名需要你去发现，请注意识别。
需要注意，不要把人名、官职名、书名等识别为地名。

以下内容为一个输入输出示例:
输入: "至德为并州刺史"
输出:
并州

请识别下面文本中的地名:
{text}
""",
    
    'OFI': """你是一个专业的古文命名实体识别专家。请仔细阅读以下古文内容，帮我识别出其中的官职名称。

常见的官职名称有：刺史、黄门侍郎、谏官、节度使、相国、太尉、右丞、齐王、陛下。这只是其中的一些例子，可能还有更多别的官职需要你去发现，请注意识别。
需要注意，不要把地名、人名、书名等识别为官职。

以下内容为一个输入输出示例:
输入: "至德为并州刺史"
输出:
刺史

请识别下面文本中的官职:
{text}
""",
    
    'BOOK': """你是一个专业的古文命名实体识别专家。请仔细阅读以下古文内容，帮我识别出其中的书名。包括：典籍名、文集名等。

常见的书名有：论、庄、易、周书、记、旧唐书。尤其注意，可能会有单个字作为书名，比如“论”代表了《论语》。请注意识别。
需要注意，不要把地名、人名、官职名等识别为书名。

以下内容为一个输入输出示例:
输入: "东都事略李筠传：筠请冯道领节度"
输出:
东都事略

请识别下面文本中的书名:
{text}
"""
}

def build_prompt(template_key: str, text: str) -> str:
    """组装prompt"""
    return PROMPT_TEMPLATES[template_key].format(text=text)

In [7]:
# Cell 5: 预测相关函数定义
def get_entities_from_model(text: str, entity_type: str, tokenizer, model) -> List[str]:
    """调用模型获取指定类型的实体"""
    prompt = build_prompt(entity_type, text)
    messages = [{"role": "user", "content": prompt}]
    
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([input_text], return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.8
    )
    
    response = tokenizer.batch_decode(
        [outputs[0][inputs.input_ids.shape[1]:]], 
        skip_special_tokens=True
    )[0]
    
    entities = []
    for line in response.strip().split('\n'):
        line = line.strip()
        if line and not line.startswith('输入') and not line.startswith('请'):
            entities.append(line)
    
    return entities

def get_predicted_labels(text: str, tokenizer, model) -> List[str]:
    """获取一个句子的所有类型的预测标签"""
    labels = ['O'] * len(text)
    
    for entity_type in ['PER', 'LOC', 'OFI', 'BOOK']:
        entities = get_entities_from_model(text, entity_type, tokenizer, model)
        
        for entity in entities:
            start = 0
            while True:
                pos = text.find(entity, start)
                if pos == -1:
                    break
                    
                if len(entity) == 1:
                    labels[pos] = f'S-{entity_type}'
                else:
                    labels[pos] = f'B-{entity_type}'
                    for i in range(pos + 1, pos + len(entity) - 1):
                        labels[i] = f'I-{entity_type}'
                    labels[pos + len(entity) - 1] = f'E-{entity_type}'
                
                start = pos + 1
    
    return labels

In [8]:
# Cell 6: 评估函数定义
def evaluate_ner_task(test_file: str, tokenizer, model, num_samples: int = 10):
    """评估NER任务的性能"""
    # 1. 读取测试数据
    sentences = []
    true_labels_list = []
    current_sentence = []
    current_labels = []
    
    with open(test_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                char, label = line.split('\t')
                current_sentence.append(char)
                current_labels.append(label)
            elif current_sentence:
                sentences.append(''.join(current_sentence))
                true_labels_list.append(current_labels)
                current_sentence = []
                current_labels = []
    
    if current_sentence:
        sentences.append(''.join(current_sentence))
        true_labels_list.append(current_labels)
    
    # 2. 随机抽样
    if num_samples < len(sentences):
        indices = random.sample(range(len(sentences)), num_samples)
        test_sentences = [sentences[i] for i in indices]
        test_true_labels = [true_labels_list[i] for i in indices]
    else:
        test_sentences = sentences
        test_true_labels = true_labels_list
    
    # 3. 获取预测结果
    all_true_labels = []
    all_pred_labels = []
    
    for sentence, true_labels in zip(test_sentences, test_true_labels):
        pred_labels = get_predicted_labels(sentence, tokenizer, model)
        
        assert len(true_labels) == len(pred_labels), \
            f"标签长度不匹配: true={len(true_labels)}, pred={len(pred_labels)}"
        
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)
        
        print("\n当前句子:", sentence)
        print("真实标签:", true_labels)
        print("预测标签:", pred_labels)
    
    # 4. 计算F1分数
    f1 = f1_score(all_true_labels, all_pred_labels, average='macro')
    return f1

In [10]:
# Cell 7: 运行评估 (这个cell可以重复运行来测试不同的样本)
test_file = "ner.txt"
f1 = evaluate_ner_task(test_file, tokenizer, model, num_samples=218)
print(f"\nMacro F1 Score: {f1}")


当前句子: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。
真实标签: ['O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'E-OFI', 'O', 'B-LOC', 'E-LOC', 'B-OFI', 'I-OFI', 'E-OFI', 'O', 'O', 'O', 'S-PER', 'O']
预测标签: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'